#### Example of running a classifier and making a heatmap for the entire slice

In [ ]:
import os
import pandas as pd
import numpy as np
import json
from panda_challenge.train_utils import get_model
from panda_challenge.inference_utils import get_slice_heatmap
import matplotlib.pyplot as plt
import matplotlib
import torch
import openslide

In [ ]:
with open('./configs/test_run_v2_config.json', 'r') as f:
    params = json.load(f)

In [ ]:
data_val = pd.read_csv('/data/personal_folders/skolchenko/panda/data_val.csv')
data_val.head(5)

In [ ]:
model = get_model(
    params['model_name'],
    **params['model_config'])
model.cuda()
checkpoint = torch.load(os.path.join(params['log_dir'], 'checkpoints/best.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
cmap = matplotlib.colors.ListedColormap([
    'black',
    'gray',
    'green',
    'yellow',
    'orange',
    'red'])

In [ ]:
idx = 12
slide_id = data_val.image_id.values[idx]
isup_grade = data_val.isup_grade.values[idx]
mask = openslide.OpenSlide(os.path.join(
    params['val_mask_dir'],
    f'{slide_id}_mask.tiff'))
image = openslide.OpenSlide(os.path.join(
    params['val_image_dir'],
    f'{slide_id}.tiff'))  
predictions = get_slice_heatmap(slide_id, params['val_image_dir'], bg_thr=0.5, model=model)

In [ ]:
mask = mask.read_region((0,0), 2, mask.level_dimensions[-1])
mask = np.asarray(mask)[..., 0]
image = image.read_region((0,0), 2, image.level_dimensions[-1])
image = np.asarray(image)[..., :3]

In [ ]:
f, ax = plt.subplots(1,3)
ax[0].imshow(mask, cmap=cmap)
ax[0].set_title('Original mask')
ax[0].set_xticks([]),
ax[0].set_yticks([])
ax[1].imshow(predictions, cmap=cmap)
ax[1].set_title('Predction heatmap')
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[2].imshow(image, cmap=cmap)
ax[2].set_title(f'Slide isup grade {isup_grade}');
ax[2].set_xticks([])
ax[2].set_yticks([]);